# Modified Gale Shapley Algorithm

In [7]:
import pandas as pd

In [8]:
syndata = pd.read_csv('synData9.csv')
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [9]:
syndata.iloc[0] #Use this to access a row
syndata.loc[0, :] #Use this to access a row
x = syndata.loc[:,"Age"] #Use this to access a column

In [10]:
syndata.columns[1]

'Number of sexual partners'

Cdoe for finding the mean. Removes any unknown values

In [86]:
def mean_for_column(data1, column):
    """
    Finds the mean for a specific column
    """
    tot = 0 
    for i in range(len(data1)):
        #print(data1.loc[i, column])
        if data1.loc[i, column] == "?":
            pass
        else:
            tot = tot + float(data1.loc[i, column])
    tot = tot/(len(data1))
    return tot


In [87]:
mean_for_column(syndata, 'Number of sexual partners')
#syndata.mean()

2.1418

Now we work on creating a compare function which is being designed to comapre all the rows of one dataset to one specific row of another dataset

In [29]:
def compare(data1, data2, row_comp):
    """
    This is a comparsion function  
    data1: First dataset for comparsion, 1 set row
    data2: Second dataset for comparsion
    row_comp: The row we want compared
    """
    #Goes for 1 section of the real data and then loops for every row of the synthesized data
    for i in range(len(data2)):
        print('Real data: ' + str(data1.loc[row_comp, 'Age']) + '. Fake data: ' +str(data2.loc[i, 'Age']))
        #Comparsion function to compare each the set real row to current fake row
        similiar(data1, data2, i, row_comp)
        if i==1:
            return

In [30]:
def similiar(data1, data2, ind, row_comp):
    """
    Computes how similiar the two datasets are 
    """
    for i in data1.columns:
       print(ind, data1.loc[row_comp,:], data2.loc[ind,:])
       print(i)
       if i ==1:
           return

In [ ]:
compare(realdata,syndata, 0)